In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib qt 

import sys
import napari
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

from consts import HERE
sys.path.append(str(HERE.parent))
from utils import  load_video_as_array
from ctko.video_processor import get_binarise_value



In [2]:
mice = [
    ("J022", "2024-09-27"),
    ("J023", "2024-09-27"),
    ("J024", "2024-10-09"),
    ("J025", "2024-09-27"),
    ("J026", "2024-10-24"),
    ("J027", "2024-10-09"),
    ("J028", "2024-10-24"),
    ("J029", "2024-10-25"),
]

In [3]:
pupil_umbrella = Path("/Volumes/MarcBusche/James/Regular2p")
idx = 6
mouse = mice[idx][0]
date = mice[idx][1]
pupil_folder = pupil_umbrella / date / mouse
mp4_paths = sorted(list(pupil_folder.glob("*.mp4")))
assert len(mp4_paths) > 0, f"No mp4s found in {pupil_folder}"
mp4_paths

[PosixPath('/Volumes/MarcBusche/James/Regular2p/2024-10-24/J028/2024-10-24_J028_pupil_2024-10-24-194429-0000.mp4'),
 PosixPath('/Volumes/MarcBusche/James/Regular2p/2024-10-24/J028/2024-10-24_J028_pupil_2024-10-24-195531-0000.mp4'),
 PosixPath('/Volumes/MarcBusche/James/Regular2p/2024-10-24/J028/2024-10-24_J028_pupil_2024-10-24-200537-0000.mp4')]

In [4]:
show_napari = True
print(mouse)

if show_napari:
    # Inspect the binary form of the video
    video = load_video_as_array(mp4_paths[0], 500)

    viewer = napari.Viewer()
    binarised = (video > get_binarise_value(mouse)).astype('int')

    viewer.add_image(video, name="Video")
    viewer.add_image(binarised, name="Binanrised")

J028


In [5]:
process = True

if process:
    shape = [layer for layer in viewer.layers if layer.name == "Shapes"][0]

    [key for key in shape.__dict__.keys() if key[0] != "_"]

    top_left =  [shape.corner_pixels[0, 1], shape.corner_pixels[0, 2]]
    bottom_right = [shape.corner_pixels[1,1], shape.corner_pixels[1,2]]

    # bottom_right[0] = bottom_right[0] +  400 - (bottom_right[0] - top_left[0])
    # bottom_right[1] = bottom_right[1] +  400 - (bottom_right[1] - top_left[1])

    # assert (bottom_right[0] - top_left[0]) == 400
    # assert (bottom_right[1] - top_left[1]) == 400


    save_path=HERE.parent / "cache" / f"{mouse}_{date}_corner_pixels.npy"
    np.save(save_path,(top_left,bottom_right))


In [6]:
print(mouse)
save_path=HERE.parent / "cache" / f"{mouse}_{date}_corner_pixels.npy"
video = load_video_as_array(mp4_paths[0], 700)
top_left,bottom_right = np.load(save_path)

cropped_video = video[:, top_left[0]: bottom_right[0], top_left[1]: bottom_right[1]]
binarised = (cropped_video > get_binarise_value(mouse)).astype(int)

diffed = np.abs(np.diff(binarised, axis=0))
viewer.add_image(binarised, name="Cropped Binarised")
viewer.add_image(diffed, name="Cropped Diffed")
viewer.add_image(cropped_video, name="Cropped Video")

J028


<Image layer 'Cropped Video' at 0x28c00e8a0>